# EDA Binary Classification with a Bank Dataset

### This EDA refers to Kaggle playground competition: "Binary Classification with a Bank Dataset" 
- Competition link: https://www.kaggle.com/competitions/playground-series-s5e8
- Main goal of this competition is to predict whether a client will subscribe to a bank term deposit.
- Submissions are evaluated using ROC AUC between the predicted value and the observed target.
- The dataset for this competition (both train and test) was generated from a deep learning model trained on the Bank Marketing Dataset dataset. Feature distributions are close to, but not exactly the same, as the original.
- Start Date - August 1, 2025
- Final Submission Deadline - August 31, 2025

### Labels details from original data set

- age: Age of the client (numeric)
- job: Type of job (categorical: "admin.", "blue-collar", "entrepreneur", etc.)
- marital: Marital status (categorical: "married", "single", "divorced")
- education: Level of education (categorical: "primary", "secondary", "tertiary", "unknown")
- default: Has credit in default? (categorical: "yes", "no")
- balance: Average yearly balance in euros (numeric)
- housing: Has a housing loan? (categorical: "yes", "no")
- loan: Has a personal loan? (categorical: "yes", "no")
- contact: Type of communication contact (categorical: "unknown", "telephone", "cellular")
- day: Last contact day of the month (numeric, 1-31)
- month: Last contact month of the year (categorical: "jan", "feb", "mar", …, "dec")
- duration: Last contact duration in seconds (numeric)
- campaign: Number of contacts performed during this campaign (numeric)
- pdays: Number of days since the client was last contacted from a previous campaign (numeric; -1 means the client was not previously contacted)
- previous: Number of contacts performed before this campaign (numeric)
- poutcome: Outcome of the previous marketing campaign (categorical: "unknown", "other", "failure", "success")
- y: The target variable, whether the client subscribed to a term deposit

In [ ]:
import numpy as np 
import pandas as pd 
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from lightgbm import LGBMRegressor, LGBMClassifier

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.manifold import TSNE
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

warnings.filterwarnings("ignore",  category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning,)
warnings.filterwarnings("ignore", category=UserWarning)


In [ ]:
train_df = pd.read_csv('/kaggle/input/playground-series-s5e8/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s5e8/test.csv')
sub_df = pd.read_csv('/kaggle/input/playground-series-s5e8/sample_submission.csv')

## 1. Basic check of data

In [ ]:
display(train_df.head(4))
display(test_df.head(4))

In [ ]:
display(train_df.info())
display(test_df.info())

In [ ]:
display("Summary for TRAIN data:",train_df.describe())
display("Summary for TEST data:",test_df.describe())

In [ ]:
display("TRAIN data", train_df.nunique())
print('\n')
display('TEST data',test_df.nunique())

### Key Observations:
- We have 7 numeric columns and 9 with categories
- month column can be mapped to numerical column if needed
- One coulmn with id in both train/test sets and additional column with labels in train set
- number of unique values different for columns age, balance, duration, campaign, pdays, previous
- min values different for train - test set for column duration
- max values different balance, campaign, previous
- pdays have two separate information - first number of days and second == -1 means the client was not previously contacted. Can be separated into two columns to separate category from numeric

### BONUS: Dimensions reduction with t-SNE

In [ ]:
X = train_df.drop(columns=['id', 'y'])
y = train_df['y']

# Separation of numeric and categorical columns
cat_cols = X.select_dtypes(include='object').columns.tolist()
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()

# Pipeline: przetwarzanie cech
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse=False), cat_cols)
])

X_processed = preprocessor.fit_transform(X)

# Dimensions reduction with t-SNE
# t-SNE is very slow for large datasets. We will check only 20k random samples
SAMPLE_SIZE = 20000

idx = np.random.choice(len(X_processed), SAMPLE_SIZE, replace=False)
X_sample = X_processed[idx]
y_sample = y.iloc[idx]

tsne = TSNE(n_components=2, perplexity=30, learning_rate=200, n_iter=1000, random_state=42)
X_tsne = tsne.fit_transform(X_sample)


# Creating plot
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x=X_tsne[:, 0],
    y=X_tsne[:, 1],
    hue=y_sample,
#    palette='coolwarm',
    s=5,
    alpha=0.6,
    linewidth=0
)
plt.title("t-SNE 2D wizualizacja danych (kolor = y)")
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.legend(title='y')
plt.grid(True)
plt.tight_layout()
plt.show()

### 2nd BONUS - Separation of pdays and additional column wuth months as numeric

In [ ]:
# we can create separate column with flag for -1 value
train_df['no_previous_contact'] = (train_df['pdays'] == -1).astype(int)
test_df['no_previous_contact'] = (test_df['pdays'] == -1).astype(int)

# We can create additional column with pdays only without -1 values
train_df['pdays_cleaned'] = train_df['pdays'].where(train_df['pdays'] != -1, np.nan) 
test_df['pdays_cleaned'] = test_df['pdays'].where(test_df['pdays'] != -1, np.nan) 

# We can create additional column with numeric months
train_df['month_as_num'] = train_df['month'].map({'jan':1,'feb':2,'mar':3,'apr':4,'may':5,'jun':6,'jul':7,'aug':8,'sep':9,'oct':10,'nov':11, 'dec':12})
test_df['month_as_num'] = test_df['month'].map({'jan':1,'feb':2,'mar':3,'apr':4,'may':5,'jun':6,'jul':7,'aug':8,'sep':9,'oct':10,'nov':11, 'dec':12})

## 2. Duplicates and missing values check

In [ ]:
print("Duplicates in TRAIN data:", train_df.duplicated().sum())
print("Duplicates in TEST data:", test_df.duplicated().sum())

In [ ]:
print("Missing values in TRAIN data:\n",train_df.isna().mean().apply(lambda x: f"{x:.2%}"))
print("\nMissing values  in TEST data:\n",test_df.isna().mean().apply(lambda x: f"{x:.2%}"))

### Key Observations:
- There is no duplicates in train/test data
- There are no missing data in train/test sets
- in case we decide to clean pdays we have missing 89,66% of data 


## 3. Train-Test drift check

### Numeric column drift

In [ ]:
for col in test_df.columns:
    if col != 'id' and test_df[col].dtype in[np.int64,np.float64]:
        sns.kdeplot(train_df[col], label='test', fill=True)
        sns.kdeplot(test_df[col], label='train', fill=True)
        plt.title(f"Drift check for Column: {col}")
        plt.legend()
        plt.show()

In [ ]:
feature ='balance'

fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
upper_limit = train_df[feature].quantile(0.99)

# Zoom 
sns.kdeplot(data=train_df, x=feature, ax=axes[0], label='train', fill=True, alpha=0.4)
sns.kdeplot(data=test_df, x=feature, ax=axes[0], label='test', fill=True, alpha=0.4)
axes[0].set_xlim(-1, upper_limit)
axes[0].set_title('Data with zoom')

# Full data
sns.kdeplot(data=train_df, x=feature, ax=axes[1], label='train', fill=True, alpha=0.4)
sns.kdeplot(data=test_df, x=feature, ax=axes[1], label='test', fill=True, alpha=0.4)
axes[1].set_title('Full data')

for ax in axes:
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
feature ='duration'

fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
upper_limit = train_df[feature].quantile(0.99)

# Zoom 
sns.kdeplot(data=train_df, x=feature, ax=axes[0], label='train', fill=True, alpha=0.4)
sns.kdeplot(data=test_df, x=feature, ax=axes[0], label='test', fill=True, alpha=0.4)
axes[0].set_xlim(-1, upper_limit)
axes[0].set_title('Data with zoom')

# Full data
sns.kdeplot(data=train_df, x=feature, ax=axes[1], label='train', fill=True, alpha=0.4)
sns.kdeplot(data=test_df, x=feature, ax=axes[1], label='test', fill=True, alpha=0.4)
axes[1].set_title('Full data')

for ax in axes:
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
feature ='campaign'

fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
upper_limit = train_df[feature].quantile(0.99)

# Zoom 
sns.kdeplot(data=train_df, x=feature, ax=axes[0], label='train', fill=True, alpha=0.4)
sns.kdeplot(data=test_df, x=feature, ax=axes[0], label='test', fill=True, alpha=0.4)
axes[0].set_xlim(-1, upper_limit)
axes[0].set_title('Data with zoom')

# Full data
sns.kdeplot(data=train_df, x=feature, ax=axes[1], label='train', fill=True, alpha=0.4)
sns.kdeplot(data=test_df, x=feature, ax=axes[1], label='test', fill=True, alpha=0.4)
axes[1].set_title('Full data')

for ax in axes:
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
feature ='pdays'

fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
upper_limit = train_df[feature].quantile(0.9)

# Zoom 
sns.kdeplot(data=train_df, x=feature, ax=axes[0], label='train', fill=True, alpha=0.4)
sns.kdeplot(data=test_df, x=feature, ax=axes[0], label='test', fill=True, alpha=0.4)
axes[0].set_xlim(-1, upper_limit)
axes[0].set_title('Data with zoom')

# Full data
sns.kdeplot(data=train_df, x=feature, ax=axes[1], label='train', fill=True, alpha=0.4)
sns.kdeplot(data=test_df, x=feature, ax=axes[1], label='test', fill=True, alpha=0.4)
axes[1].set_title('Full data')

for ax in axes:
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
feature ='previous'

fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
upper_limit = train_df[feature].quantile(0.999)

# Zoom 
sns.kdeplot(data=train_df, x=feature, ax=axes[0], label='train', fill=True, alpha=0.4)
sns.kdeplot(data=test_df, x=feature, ax=axes[0], label='test', fill=True, alpha=0.4)
axes[0].set_xlim(-1, upper_limit)
axes[0].set_title('Data with zoom')

# Full data
sns.kdeplot(data=train_df, x=feature, ax=axes[1], label='train', fill=True, alpha=0.4)
sns.kdeplot(data=test_df, x=feature, ax=axes[1], label='test', fill=True, alpha=0.4)
axes[1].set_title('Full data')

for ax in axes:
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()

### Categorical columns drift check

In [ ]:
def plot_category_drift(feature):
    pd.concat([
        train_df[feature].value_counts(normalize=True).rename("train"),
        test_df[feature].value_counts(normalize=True).rename("test")
    ], axis=1).plot(kind="bar", title=f"Category drift: {feature}")

columns = [ 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact','month', 'poutcome'  ]

for col in columns:
    plot_category_drift(col)

### Key Observations:
- visible drift for column 'previous'
- no clear drift between train and test data for rest of numeric columns
- no clear drift between train and test data for categorical columns

## 4. Correlation check for train and test data

In [ ]:
sns.heatmap(train_df[['age', 'balance','day', 'duration','campaign', 'pdays','previous', 'month_as_num', 'pdays_cleaned']].corr(),
            annot = True, cmap='coolwarm')

In [ ]:
sns.heatmap(test_df[['age', 'balance','day', 'duration','campaign', 'pdays','previous','month_as_num', 'pdays_cleaned']].corr(), 
            annot = True, cmap='coolwarm')

### Key Observations:
- Only pdays and previous columns show strong correlation
- possible weak negative correlation between new columns - month_as_num and pdays_cleaned

## 5. Check of 'y' in each column for train data

### Category columns

In [ ]:
sns.countplot(data=train_df, x='y')
round(train_df['y'].value_counts(normalize=True)*100,2)

In [ ]:
columns = [ 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',  'poutcome', 
            'month_as_num', 'no_previous_contact']
for col in columns:
    train_df.groupby([col,'y']).size().unstack().plot(kind='bar', stacked=True, title=col)
    plt.show()
    print('Percentage summary:')
    display((pd.crosstab(train_df[col], train_df["y"], normalize='index') * 100).round(1))
    print('Quantitative summary:')
    display((pd.crosstab(train_df[col], train_df["y"])))

### Key Observations:
- We have 2 gropus in label column with split 88% 12% - strong unbalance of classes
- Each categorical column show caategories with different ratio of 0 / 1 labels and have potential to be used in classification


### Numerical columns

In [ ]:
columns = [ 'age','balance', 'day', 'duration', 'campaign', 'pdays', 'previous',  'pdays_cleaned', 'month_as_num']

for feature in columns:
    plt.figure(figsize=(8, 4))
    sns.kdeplot(data=train_df[train_df['y'] == 0], x=feature, label='y = 0', fill=True, alpha=0.4)
    sns.kdeplot(data=train_df[train_df['y'] == 1], x=feature, label='y = 1', fill=True, alpha=0.4)
    plt.title(f'KDE for {feature}')
    #plt.xscale('log')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

### Key Observations:
- Most of columns show differences in density for labels 0 an 1 and have good potential to be used for classification - age, day, duration, pdays_cleaned, month_as_num
- month column seems to not show cyclical behaviour so it can be probably used as categorical column
- day seems to have no cyclical behaviour so it can be also consider as categorical however 31 categories can be quite large number - to be checked
- new column pdays_cleaned show differences between 0 and 1 but it consider about 11% of data, rest of them is -1
- some plots are not so godd visible so they can be analysed in more details like previous

### Additional plots for more details

In [ ]:
feature ='balance'

fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
upper_limit = train_df[feature].quantile(0.99)

# Zoom
sns.kdeplot(data=train_df[train_df['y'] == 0], x=feature, ax=axes[0], label='y = 0', fill=True, alpha=0.4)
sns.kdeplot(data=train_df[train_df['y'] == 1], x=feature, ax=axes[0], label='y = 1', fill=True, alpha=0.4)
axes[0].set_xlim(0, upper_limit)
axes[0].set_title(f'Zoom for {feature}')

# Full
sns.kdeplot(data=train_df[train_df['y'] == 0], x=feature, ax=axes[1], label='y = 0', fill=True, alpha=0.4)
sns.kdeplot(data=train_df[train_df['y'] == 1], x=feature, ax=axes[1], label='y = 1', fill=True, alpha=0.4)
axes[1].set_title(f'Full data for {feature}')

for ax in axes:
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
feature ='duration'

fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
upper_limit = train_df[feature].quantile(0.995)

# Zoom
sns.kdeplot(data=train_df[train_df['y'] == 0], x=feature, ax=axes[0], label='y = 0', fill=True, alpha=0.4)
sns.kdeplot(data=train_df[train_df['y'] == 1], x=feature, ax=axes[0], label='y = 1', fill=True, alpha=0.4)
axes[0].set_xlim(0, upper_limit)
axes[0].set_title(f'Zoom for {feature}')

# Full
sns.kdeplot(data=train_df[train_df['y'] == 0], x=feature, ax=axes[1], label='y = 0', fill=True, alpha=0.4)
sns.kdeplot(data=train_df[train_df['y'] == 1], x=feature, ax=axes[1], label='y = 1', fill=True, alpha=0.4)
axes[1].set_title(f'Full data for {feature}')

for ax in axes:
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
feature ='campaign'

fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
upper_limit = train_df[feature].quantile(0.97)

# Zoom
sns.kdeplot(data=train_df[train_df['y'] == 0], x=feature, ax=axes[0], label='y = 0', fill=True, alpha=0.4)
sns.kdeplot(data=train_df[train_df['y'] == 1], x=feature, ax=axes[0], label='y = 1', fill=True, alpha=0.4)
axes[0].set_xlim(0, upper_limit)
axes[0].set_title(f'Zoom  for {feature}')

# Full
sns.kdeplot(data=train_df[train_df['y'] == 0], x=feature, ax=axes[1], label='y = 0', fill=True, alpha=0.4)
sns.kdeplot(data=train_df[train_df['y'] == 1], x=feature, ax=axes[1], label='y = 1', fill=True, alpha=0.4)
axes[1].set_title(f'Full data for {feature}')

for ax in axes:
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
feature ='previous'

fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
upper_limit = train_df[feature].quantile(0.995)

# Zoom
sns.kdeplot(data=train_df[train_df['y'] == 0], x=feature, ax=axes[0], label='y = 0', fill=True, alpha=0.4)
sns.kdeplot(data=train_df[train_df['y'] == 1], x=feature, ax=axes[0], label='y = 1', fill=True, alpha=0.4)
axes[0].set_xlim(0, upper_limit)
axes[0].set_title(f'Zoom for {feature}')

# Full
sns.kdeplot(data=train_df[train_df['y'] == 0], x=feature, ax=axes[1], label='y = 0', fill=True, alpha=0.4)
sns.kdeplot(data=train_df[train_df['y'] == 1], x=feature, ax=axes[1], label='y = 1', fill=True, alpha=0.4)
axes[1].set_title(f'Full data for {feature}')

for ax in axes:
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()

### Key Observations:
- Balance column show small differences in density - can be verified with feature importance
- duration column show quite good differences in density - it is good potential for classification
- campaign column seems to have no bigger differences -  can be verified with feature importance
- for column previous KDE seems to show much different behaviour and seems to have a very good potential for classification

## 6. Outliners for numerical columns

In [ ]:
columns = [ 'age','balance', 'day', 'duration', 'campaign', 'pdays', 'previous',  'pdays_cleaned', 'month_as_num']

for col in columns:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x='y', y=col, data=train_df)
    plt.title(f'Boxplot of {col} by y label column')
    plt.show()

In [ ]:
## t_SNE lub PCA ???? 

In [ ]:
sns.countplot(data=train_df,  x='housing', hue='y')

In [ ]:
sns.boxplot(data=train_df, x='education', y='pdays_cleaned')

In [ ]:
sns.boxplot(data=train_df, x='marital', y='age', hue='y')

In [ ]:
sns.pairplot(train_df[[ 'age','balance', 'pdays_cleaned','y']], hue='y') 

🔧 1. Przekształcenia kolumn kategorycznych
a) One-Hot Encoding / Target Encoding
Dla zmiennych takich jak: job, marital, education, contact, month, poutcome.

Jeśli korzystasz z modeli drzewiastych (np. XGBoost, LightGBM), możesz użyć Target Encoding.

Dla regresji logistycznej czy SVM – One-Hot Encoding.

b) Grupowanie kategorii
job: Możesz pogrupować zawody wg dochodu, statusu społecznego lub stabilności zatrudnienia.

education: Połączyć unknown z primary lub stworzyć grupę "unknown" osobno – sprawdzić korelację z targetem.

month: Zamienić na numer miesiąca (jan = 1 itd.) i/lub pogrupować na kwartały lub sezony (wiosna/lato/jesień/zima).

poutcome: Bardzo ważna zmienna – warto zostawić jako osobną kolumnę, ale też stworzyć zmienną binarną typu: prev_success = poutcome == 'success'.

🧮 2. Przekształcenia kolumn liczbowych
a) Standaryzacja / Normalizacja
Dla modeli wrażliwych na skalę (regresja logistyczna, SVM) warto przeskalować: age, balance, duration, campaign, pdays, previous.

b) Transformacje logarytmiczne / winsoryzacja
balance, duration, pdays, previous mogą mieć rozkład z ogonami – warto sprawdzić wykresy. Spróbuj:

log(1 + balance), log(1 + duration), log(1 + previous)

Winsoryzacja (obcięcie ekstremalnych wartości)

🧠 3. Tworzenie nowych cech (Feature Engineering)
a) Interakcje między zmiennymi
age * duration, job + education, contact * month, poutcome * previous – interakcje mogą ujawniać niuanse kampanii.

Stwórz kolumnę recently_contacted = pdays != -1 jako cecha binarna.

b) Grupowanie wieku
age_group = 'young' (<=30) / 'middle' (31–60) / 'senior' (>60) – może mieć różny wpływ na decyzje kredytowe.

c) Zmienna sezonowa
is_summer = month in ['jun', 'jul', 'aug'] – kampanie w wakacje mogą mieć inną skuteczność.

d) Długość kontaktu / skuteczność kontaktu
long_contact = duration > X (np. 120 sekund)

campaign_efficiency = previous / (1 + campaign) – czy wiele kontaktów wcześniej przynosiło efekt?

🔍 4. Sprawdzenie korelacji / ważności cech
Wykorzystaj:

Feature Importance z modelu drzewiastego.

Permutation Importance

SHAP values – do zrozumienia wpływu cech na predykcję.

🧪 5. Eksperymenty i walidacja
a) Sprawdź:
Czy duration nie przecieka informacji? (jeśli znana tylko po kampanii – może trzeba ją pominąć).

Czy pdays == -1 to tylko brak kontaktu, czy też dodatkowa informacja o “świeżym” kliencie?

Jaki jest rozkład targetu y – czy masz problem imbalance? Jeśli tak, rozważ:

SMOTE, undersampling majority class, class_weight.

✅ Przykładowe cechy do przetestowania:
Nowa kolumna	Opis
age_group	kategoryczna: young / middle / senior
log_balance	log(1 + balance)
contacted_before	binary: pdays != -1
season	categorical: winter/spring/summer/fall
contact_efficiency	previous / (1 + campaign)
long_contact	binary: duration > 120
education_known	binary: education != 'unknown'

1. Czasowe i kampanijne zależności
⏱️ „Długość kontaktu” / „liczba kontaktów”
kontakt_avg_time = duration / campaign – średni czas kontaktu na jeden kontakt

📉 Trend kontaktów
delta_contact = previous - campaign – czy liczba kontaktów wzrosła czy spadła

has_previous_contact = (pdays != 999).astype(int) – flaga, czy kontakt był wcześniej

📅 Sezonowość
Zmienna month – zakoduj ją jako int (np. Jan = 1) i dodaj zmienną:

is_q4_campaign = month.isin(['oct', 'nov', 'dec']) – może zimą skuteczność spada?

🧑‍💼 2. Zachowanie klienta i profil demograficzny
💳 Zobowiązania finansowe
loan_sum = (housing == 'yes') + (loan == 'yes') – liczba aktywnych pożyczek

is_deep_debt = (balance < 0) & (loan_sum > 1) – mocno zadłużony

🧠 Poziom edukacji + zawód
edu_job = education + "_" + job – np. "tertiary_admin"

Można zakodować i użyć jako cechy (one-hot lub target encoding)

👫 Małżeństwo vs. wiek
is_young_single = (age < 30) & (marital == 'single')

is_old_married = (age > 60) & (marital == 'married')

📞 3. Komunikacja i kanał kontaktu
🔔 Efektywność kanału kontaktu
contact_success_rate = campaign / (duration + 1) – ile kontaktów na minutę

🛠️ Rodzaj komunikacji + sukces poprzedni
channel_prev_success = contact + "_" + poutcome



🧪 Narzędzia do wykrywania nieliniowych zależności:
pd.plotting.scatter_matrix() – szybki rzut oka

sns.pairplot() – dla małej liczby kolumn

sklearn.feature_selection.mutual_info_classif(X, y) – mierzy nieliniową zależność

1. Numeryczne ↔ Numeryczne
Pairplot (sns.pairplot)
Kilka zmiennych naraz – przegląd zależności i gęstości
sns.pairplot(df, hue='label')  # opcjonalnie hue dla klasy binarnej

Scatterplot z hue / style / size
Super do odkrywania nieliniowych relacji
sns.scatterplot(data=df, x='age', y='balance', hue='loan', style='marital')

Heatmap korelacji (sns.heatmap)
Pokazuje mocne lub słabe powiązania liniowe
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm')

2. Kategoryczne ↔ Numeryczne
Boxplot (sns.boxplot)
Widać mediany, rozrzut i outliery
sns.boxplot(data=df, x='education', y='balance')

Violinplot (sns.violinplot)
To samo co boxplot, ale z KDE – pokazuje lepiej gęstości
sns.violinplot(data=df, x='job', y='age', hue='y', split=True)

Swarmplot (sns.swarmplot)
Każdy punkt osobno – dobry na małych zbiorach
sns.swarmplot(data=df, x='marital', y='balance')

Barplot (sns.barplot)
Średnia wartość cechy numerycznej dla kategorii (np. średni balance dla job)
sns.barplot(data=df, x='job', y='balance')

 3. Kategoryczne ↔ Kategoryczne
Heatmap cross-tab (pd.crosstab + sns.heatmap)
Ile wystąpień danego połączenia
ct = pd.crosstab(df['job'], df['marital'])
sns.heatmap(ct, annot=True, fmt='d', cmap='Blues')

Countplot (sns.countplot)
Liczba obserwacji w każdej kategorii + hue np. y
sns.countplot(data=df, x='education', hue='y')

4. Numeryczne ↔ Target binarny
KDE Plot (sns.kdeplot)
Rozkład cechy w dwóch klasach
sns.kdeplot(data=df[df['y'] == 0], x='age', label='No Loan')
sns.kdeplot(data=df[df['y'] == 1], x='age', label='Loan Taken')

Histogram + hue
sns.histplot(data=df, x='balance', hue='y', bins=30, kde=True, stat='density')
🔹 5. Dodatkowe / Interaktywne
✅ FacetGrid
Podzielone wykresy np. według marital i education
g = sns.FacetGrid(df, col="marital", row="education", hue="y")
g.map(sns.kdeplot, "age", fill=True)

jointplot (sns.jointplot)
Scatter + marginesowe rozkłady
sns.jointplot(data=df, x='balance', y='duration', hue='y', kind='kde')



## 6. Detailed check of categorical columns properties

In [ ]:
sns.catplot(
    data=train_df[train_df['Personality'].isin(['Introvert', 'Extrovert'])],  
    x="Stage_fear",
    hue="Personality",
    col="Drained_after_socializing",
    kind="count",
    height=4,
    aspect=1
)
plt.suptitle("Rozkład StageFear wg osobowości", y=1.05)
plt.show()

In [ ]:
pd.crosstab([train_df['Personality'],train_df["Stage_fear"]],train_df["Drained_after_socializing"]  )

### Key Observations:
- Stage_fear and Drained_after_socializing columns seems to be strong indicator of classification
- In each column there is small representation of opposite group prefference (Extrovert preffers No & No, but there is small group of introverts with same prefferences)
- It is very rare situation to have Yes - No and No - Yes answers. It is very specific minor group
- It may be resonable to impute data Yes when other column is Yes and oposite for No, for such rare cases


####  Below we check what would be result of imputation Yes to Yes and No to No for both ways

In [ ]:
cat_cols1 = ['Stage_fear','Drained_after_socializing']
all_train_df=train_df[['Stage_fear','Drained_after_socializing','Personality']].copy()
all_train_df[cat_cols1]=all_train_df[cat_cols1].fillna('Missing').astype(str)
display(pd.crosstab([all_train_df['Personality'],all_train_df["Stage_fear"]],all_train_df["Drained_after_socializing"]  ))


help_train_df = train_df[['Stage_fear','Drained_after_socializing','Personality']].copy()
help_train_df['Stage_fear'] = help_train_df['Stage_fear'].mask(help_train_df['Stage_fear'].isna() & help_train_df['Drained_after_socializing']
                                            .notna(), help_train_df['Drained_after_socializing'])
help_train_df['Drained_after_socializing'] = help_train_df['Drained_after_socializing'].mask(help_train_df['Drained_after_socializing']
                                            .isna() & help_train_df['Stage_fear'].notna(), help_train_df['Stage_fear'])
help_train_df[cat_cols1]=help_train_df[cat_cols1].fillna('Missing').astype(str)

display(pd.crosstab(help_train_df['Stage_fear'],help_train_df['Personality']))

### Key Observations:
- There is only 39 missing values for both categorical columns and for total 18524 rows it seems to be a good imputation stratego - to be tested

## 7. Detailed check of No-No / Yes-Yes data regarding Introverts & Extroverts groups

In [ ]:
train_introvert_df = train_df[train_df['Personality']=='Introvert']
train_extrovert_df = train_df[train_df['Personality']=='Extrovert']

In [ ]:
introvert_no_no_df = train_df[(train_df["Stage_fear"]=='No') & (train_df["Drained_after_socializing"]=='No') & (train_df['Personality']=='Introvert')]
extrovert_yes_yes_df = train_df[(train_df["Stage_fear"]=='Yes') & (train_df["Drained_after_socializing"]=='Yes') & (train_df['Personality']=='Extrovert')]

In [ ]:
for col in columns:
    if train_df[col].dtype in[np.int64,np.float64]:
        sns.kdeplot(introvert_no_no_df[col], label='introvert with no-no', fill=True)
        sns.kdeplot(train_introvert_df[col], label='train_df introvert', fill=True)
        sns.kdeplot(train_extrovert_df[col], label='train_df extrovert', fill=True)
        plt.legend()
        plt.show()

In [ ]:
for col in columns:
    if train_df[col].dtype in[np.int64,np.float64]:
        sns.kdeplot(extrovert_yes_yes_df[col], label='extrovert with yes-yes', fill=True)
        sns.kdeplot(train_introvert_df[col], label='train_df introvert', fill=True)
        sns.kdeplot(train_extrovert_df[col], label='train_df extrovert', fill=True)
        plt.legend()
        plt.show()

In [ ]:
for col in columns:
    if train_df[col].dtype in[np.int64,np.float64]:
        sns.kdeplot(extrovert_yes_yes_df[col], label='extrovert with yes-yes', fill=True)
        sns.kdeplot(train_introvert_df[col], label='train_df introvert', fill=True)
        sns.kdeplot(train_extrovert_df[col], label='train_df extrovert', fill=True)
        sns.kdeplot(introvert_no_no_df[col], label='introvert with no-no', fill=True)
        plt.legend()
        plt.show()

### Key Observations:
- It seems like main problem of this classification. No-No answer is typical for Extroverts but some Introverts have same prefferences like No-No and has similar values for other colums (Post_frequency, Going_outsice...) like Extroverts
- It will be extremly hard for classifiers to deal with such situation. There is some space in overlaping area between yes-yes and no-no groups and it can be marked with 0 and 1 giving clear information to model that is border condition. - Can be tested 

In [ ]:
for col in test_df.columns:
    if col != 'id' and test_df[col].dtype in[np.int64,np.float64]:
        sns.kdeplot(introvert_no_no_df[col], label='introvert with no-no', fill=True)
        sns.kdeplot(extrovert_yes_yes_df[col], label='extrovert with yes-yes', fill=True)

        plt.legend()
        plt.show()

### Markers to be tested:
- 'Time_spent_Alone' == 4
- 'Social_event_attendance' == 3
- 'Going_outside' ==3
- 'Friends_circle_size' == 5
- 'Post_frequency' == 3

## 8. Missing values - looking for signals

In [ ]:
excluded_cols = ['id', 'Personality']
all_columns = train_df.columns
for col in all_columns:
    if col not in excluded_cols:
        train_df[col + '_MISS'] = train_df[col].notna().astype(int)

In [ ]:
columns = ['Time_spent_Alone_MISS','Stage_fear_MISS', 'Social_event_attendance_MISS', 'Going_outside_MISS','Drained_after_socializing_MISS', 
           'Friends_circle_size_MISS','Post_frequency_MISS']
for col in columns:
    train_df.groupby([col,'Personality']).size().unstack().plot(kind='bar', stacked=True, title=col)
    result = pd.crosstab(train_df[col],train_df['Personality'], normalize='index')*100
    chi2, p, _, _ = chi2_contingency(result)
    print(f"Chi2 = {chi2:.3f}, p-value = {p:.4f} Column: {col} ")

### Key Observations:
- Columns Stage_fear_MISS and Drained_after_socializing_MISS have p-value lower then 0.05 so they can be considered as potential signal for different distribution of Introverts/Extroverts

## 9. Missing values - total number of non-missing data

In [ ]:
train_df['not_MISS_total'] = train_df[columns].sum(axis=1)
train_df.groupby(['not_MISS_total','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Data without NaN values')
pd.crosstab(train_df['not_MISS_total'],train_df['Personality'], normalize='index')*100

### Key Observations:
- When number of missing values for one person increase it is observed that percentage of introverts in such grup increases too. 

## 10. Advanced Data Imputation 

In [ ]:
num_cols = ['Time_spent_Alone', 'Social_event_attendance', 'Going_outside', 'Friends_circle_size','Post_frequency']    
cat_cols = ['Stage_fear', 'Drained_after_socializing']     

cat_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

train_df[cat_cols] = cat_encoder.fit_transform(train_df[cat_cols])
num_imputer = IterativeImputer(estimator=LGBMRegressor(n_estimators=500, learning_rate=0.03, max_depth=6, subsample=0.8, colsample_bytree=0.8, verbosity=-1),
                               max_iter=10, random_state=42)

train_df[num_cols] = num_imputer.fit_transform(train_df[num_cols])
cat_imputer = IterativeImputer(estimator=LGBMClassifier(n_estimators=500, learning_rate=0.03, max_depth=6, subsample=0.8, colsample_bytree=0.8, class_weght='balanced', verbosity=-1),
                               max_iter=10, random_state=42)

train_df[cat_cols] = cat_imputer.fit_transform(train_df[cat_cols])

columns = ['Time_spent_Alone', 'Social_event_attendance', 'Going_outside', 'Friends_circle_size','Post_frequency']
train_df[columns]=train_df[columns].round().astype(int)


## 11. Realations check between columns

### Time_spent_Alone / Going_outside

In [ ]:
train_df['Time_Alone_dev_Outside'] = train_df['Time_spent_Alone'] / train_df['Going_outside']
train_df['Time_Alone_dev_Outside']=train_df['Time_Alone_dev_Outside'].round(2).astype(float)
train_df.groupby(['Time_Alone_dev_Outside','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

In [ ]:
def Time_Alone_dev_Outside (x):
    try:
        x=float(x)
        if x <= 1:
            return 0
        elif x > 1 and x < 2:
            return 1
        elif x >= 2 and x < 100:
            return 2
        else:
            return 3
    except ValueError:
        return 3

train_df['Time_Alone_dev_Outside']=train_df['Time_Alone_dev_Outside'].apply(Time_Alone_dev_Outside).astype('Int64')
train_df.groupby(['Time_Alone_dev_Outside','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')
pd.crosstab(train_df["Time_Alone_dev_Outside"],train_df['Personality'],normalize='index')*100 

### Key Observations:
- If columns Time_spent_Alone / Going_outside are devided we can observed quite good separation of data
- To reduce number of features we can use clustering or write simple function, each group have different ratio
- Can be check as additional feature to recognise groups

## Social_event_attendance / Post_frequency

In [ ]:
train_df['Social_dev_Post'] = train_df['Social_event_attendance'] / train_df['Post_frequency']
train_df['Social_dev_Post']=train_df['Social_dev_Post'].round(2).astype(float)
train_df.groupby(['Social_dev_Post','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

In [ ]:
## Alternatywne grupowanie
train_df['Social_dev_Post'] = train_df['Social_event_attendance'] / train_df['Post_frequency']
train_df['Social_dev_Post']=train_df['Social_dev_Post'].round(2).astype(float)

def Social_dev_Post (x):
    try:
        x=float(x)
        if x == 0:
            return 0
        elif x > 0 and x < 0.33:
            return 1
        elif x == 0.33:
            return 2
        elif x > 0.33 and x < 0.5:
            return 1
        elif x == 0.5:
            return 4
        elif x > 0.5 and x < 0.67:
            return 1
        elif x == 0.67:
            return 4
        elif x > 0.67 and x < 1:
            return 1
        elif x == 1:
            return 4
        elif x > 1 and x < 1.5:
            return 1
        elif x == 1.5:
            return 4
        elif x > 1.5 and x < 2:
            return 1
        elif x == 2:
            return 4
        elif x > 2 and x < 3:
            return 1
        elif x == 3:
            return 2
        elif x > 3 and x < 100:
            return 1
        else:
            return 0
    except ValueError:
        return 0

train_df['Social_dev_Post']=train_df['Social_dev_Post'].apply(Social_dev_Post).astype('Int64')
train_df.groupby(['Social_dev_Post','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')
pd.crosstab(train_df["Social_dev_Post"],train_df['Personality'], normalize='index')*100

### Key Observations:
- If columns Social_event_attendance / Post_frequency are devided we can observed quite good separation of data in fixed points
- To reduce number of features we can use clustering or write simple function, each group have different ratio
- Can be check as additional feature to recognise groups

## Going_outside * Friends_circle_size

In [ ]:
train_df['Outside_mult_Friends'] = train_df['Going_outside'] * train_df['Friends_circle_size']
train_df.groupby(['Outside_mult_Friends','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

In [ ]:
def Outside_mult_Friends (x):
    try:
        x=float(x)
        if x <= 11:
            return 0
        elif x > 11 and x <= 15:
            return 1
        elif x > 15 and x < 400:
            return 2
        else:
            return 2
    except ValueError:
        return 2

train_df['Outside_mult_Friends']=train_df['Outside_mult_Friends'].apply(Outside_mult_Friends).astype('Int64')
train_df.groupby(['Outside_mult_Friends','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')
pd.crosstab(train_df["Outside_mult_Friends"],train_df['Personality'], normalize='index')*100

### Key Observations:
- If columns Going_outside * Friends_circle_size are multiplied we can observed quite good separation of data 
- To reduce number of features we can use clustering or write simple function, each group have different ratio
- Can be check as additional feature to recognise groups

## Going_outside - Post_frequency

In [ ]:
train_df['Going_sub_Post']=train_df['Going_outside'] - train_df['Post_frequency']
train_df.groupby(['Going_sub_Post','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

In [ ]:
def Outside_mult_Friends (x):
    try:
        x=float(x)
        if x <= -6:
            return 0
        elif x == -5 or x== -4 or x==4:
            return 1
        elif x == -3 or x== 3:
            return 2
        elif x == -2 or x== 2:
            return 3
        elif x == -1 or x== 0 or x==1:
            return 4
        elif x >= 5:
            return 5
        else:
            return 6
    except ValueError:
        return 6

train_df['Going_sub_Post']=train_df['Going_sub_Post'].apply(Outside_mult_Friends).astype('Int64')
train_df.groupby(['Going_sub_Post','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')
pd.crosstab(train_df["Going_sub_Post"],train_df['Personality'], normalize='index')*100

### Key Observations:
- If columns Going_outside - Post_frequency are subtracted from each other we can observed quite good separation of data 
- To reduce number of features we can use clustering or write simple function, each group have different ratio
- Can be check as additional feature to recognise groups
- group 0 is still not pure group

## 12. Realations check between columns - Other 

## Columns subtraction

In [ ]:
train_df['subtraction']=train_df['Friends_circle_size'] - train_df['Post_frequency']
train_df.groupby(['subtraction','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

In [ ]:
train_df['subtraction']=train_df['Going_outside'] - train_df['Post_frequency']
train_df.groupby(['subtraction','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

In [ ]:
train_df['subtraction']=train_df['Going_outside'] - train_df['Friends_circle_size']
train_df.groupby(['subtraction','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

In [ ]:
train_df['subtraction']=train_df['Social_event_attendance'] - train_df['Post_frequency']
train_df.groupby(['subtraction','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

In [ ]:
train_df['subtraction']=train_df['Social_event_attendance'] - train_df['Friends_circle_size']
train_df.groupby(['subtraction','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

In [ ]:
train_df['subtraction']=train_df['Social_event_attendance'] - train_df['Going_outside']
train_df.groupby(['subtraction','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

In [ ]:
train_df['subtraction']=train_df['Time_spent_Alone'] - train_df['Post_frequency']
train_df.groupby(['subtraction','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

In [ ]:
train_df['subtraction']=train_df['Time_spent_Alone'] - train_df['Friends_circle_size']
train_df.groupby(['subtraction','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

In [ ]:
train_df['subtraction']=train_df['Time_spent_Alone'] - train_df['Going_outside']
train_df.groupby(['subtraction','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

In [ ]:
train_df['subtraction']=train_df['Time_spent_Alone'] - train_df['Social_event_attendance']
train_df.groupby(['subtraction','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

## Column summing

In [ ]:
train_df['Summary']=train_df['Friends_circle_size'] + train_df['Post_frequency']
train_df.groupby(['Summary','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

In [ ]:
train_df['Summary']=train_df['Going_outside'] + train_df['Friends_circle_size']
train_df.groupby(['Summary','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

In [ ]:
train_df['Summary']=train_df['Going_outside'] + train_df['Post_frequency']
train_df.groupby(['Summary','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

In [ ]:
train_df['Summary']=train_df['Social_event_attendance'] + train_df['Post_frequency']
train_df.groupby(['Summary','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

In [ ]:
train_df['Summary']=train_df['Social_event_attendance'] + train_df['Going_outside']
train_df.groupby(['Summary','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

## Other column multiplications and divisions

In [ ]:
train_df['Time_alona_outside'] = train_df['Going_outside'] / train_df['Post_frequency']
train_df.groupby(['Time_alona_outside','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

train_df['Time_alona_outside'] = train_df['Social_event_attendance'] / train_df['Post_frequency']
train_df.groupby(['Time_alona_outside','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

train_df['Time_alona_outside'] = train_df['Time_spent_Alone'] / train_df['Post_frequency']
train_df.groupby(['Time_alona_outside','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

train_df['Time_alona_outside'] = train_df['Time_spent_Alone'] / train_df['Friends_circle_size']
train_df.groupby(['Time_alona_outside','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

train_df['Time_alona_outside'] = train_df['Time_spent_Alone'] / train_df['Social_event_attendance']
train_df.groupby(['Time_alona_outside','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

train_df['Time_alona_outside'] = train_df['Time_spent_Alone'] / train_df['Going_outside']
train_df.groupby(['Time_alona_outside','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

train_df['Time_alone_friends_circle'] = train_df['Time_spent_Alone'] / train_df['Friends_circle_size']
train_df.groupby(['Time_alone_friends_circle','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')


train_df['Social_event_outside'] = train_df['Going_outside'] * train_df['Social_event_attendance']
train_df.groupby(['Social_event_outside','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

train_df['Social_event_outside'] = train_df['Social_event_attendance'] * train_df['Friends_circle_size']
train_df.groupby(['Social_event_outside','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

train_df['Social_event_outside'] = train_df['Social_event_attendance'] * train_df['Post_frequency']
train_df.groupby(['Social_event_outside','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

train_df['Social_event_outside'] = train_df['Going_outside'] * train_df['Friends_circle_size']
train_df.groupby(['Social_event_outside','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')


train_df['Social_event_outside'] = train_df['Going_outside'] * train_df['Post_frequency']
train_df.groupby(['Social_event_outside','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')

train_df['Social_event_outside'] = train_df['Friends_circle_size'] * train_df['Post_frequency']
train_df.groupby(['Social_event_outside','Personality']).size().unstack().plot(kind='bar', stacked=True, title='Summary')


### Key Observations:
- For checked interactions there is no pure separation between Extroverts and Introverts
- Some additional features can be created to be tested, if there is improvement in classification
